# Demo No. 11 - Radial velocities fitting.

Following 2 demos will illustrate, how to proceed in case of the binary system with available radial velocities and photometric data. In this demo, we will focus on radial velocity data fitting using couple of methods.


In [16]:
import numpy as np
import astropy.units as u

from elisa.conf import config
from elisa.analytics import RVData
from elisa.analytics import BinarySystemAnalyticsTask


# setting up a custom logging config to prevent unreasonably long log messages during fit
config.LOG_CONFIG ='jupyter_logging.json'
config.set_up_logging()  

As a start, we will create datasets containing our observations:

In [17]:
rv_primary = RVData.load_from_file(filename='demo_data/rv_data/rv1.dat', 
                                   x_unit=u.d, y_unit=u.km/u.s)

rv_secondary = RVData.load_from_file(filename='demo_data/rv_data/rv2.dat', 
                                     x_unit=u.d, y_unit=u.km/u.s)

As a next step, we will create an `BinarySystemAnalyticsTask`:

In [18]:
task = BinarySystemAnalyticsTask(radial_velocities={'primary': rv_primary, 'secondary': rv_secondary})

This `task` instance now contains `rv_fit` attribute in which our radial velocity fitting methods can be accessed. 

Now we have to define a dictionary containing starting values of a fit. The list of necessary fit parameters depends on which fit approach is used. In `standard` approach, component masses are fitted. On the other side, `community` approach enables to fit the model using mass ratio and parameter a sin(i) which is much more suitable for radial velocity curve fit. List of necessary fit parameters for each approach can be accessed here:

In [19]:
task.rv_fit.FIT_PARAMS_COMBINATIONS

{'standard': ['p__mass',
  's__mass',
  'inclination',
  'eccentricity',
  'argument_of_periastron',
  'gamma',
  'period',
  'primary_minimum_time'],
 'community': ['mass_ratio',
  'asini',
  'eccentricity',
  'argument_of_periastron',
  'gamma',
  'period',
  'primary_minimum_time']}

In case of radial velocities fitting, it is much more suitable to use `community` approach which will in general remove degeneracies in a fittimg model. 

Lets finally define our starting parameters. Starting parameters are listed in form of dictionary that contain name of the variable as a key and values contains dictionary characterizing starting value of the parameter, status of the parameter (`fixed`: True/False), the boundaries of the fitted parameter defined by `min`, `max` values and the unit in astropy format.

In [20]:
rv_initial = {
    'eccentricity': {
        'value': 0.03,
        'fixed': False,
        'min': 0.0,
        'max': 0.05
    },
    'asini': {
        'value': 12,
        'fixed': False,
        'min': 7.0,
        'max': 15,
        'unit': u.solRad
    },
    'mass_ratio': {
        'value': 1.0,
        'fixed': False,
        'min': 0.9,
        'max': 1.2
    },
    'argument_of_periastron': {
        'value': 180,
        'fixed': False,
        'min': 0,
        'max': 360,
        'unit': u.deg
    },
    'gamma': {
        'value': -25.0,
        'fixed': False,
        'min': -10.0,
        'max': -30.0,
        'unit': u.km/u.s
    },
    'period': {
        'value': 2.47028376,
        'fixed': True,
    },
    'primary_minimum_time': {
        'value': 54953.8691,
        'fixed': False,
        'min': 54953.800,
        'max': 54954.000,
        'unit': u.d
    }
}

At this point, we can perform actual fit. In current version of Elisa, two fitting methods are available, least-squares and Markov chain Monte Carlo. Lets start with the first one:

In [21]:
fit_params = task.rv_fit.fit(x0=rv_initial, method='least_squares')